# Music Recommender

## Functions for Spotipy

#### Actions
If song not in scraped top 100 dataframe...
1. get songs and artists from spotipy playlist and conduct same check
    1. fetch songs, artists, audiofeatures
    2. make dataframe
    3. check
2. if song in spotify playlist, recommend another song from the playlist
    1. get random song
    2. check if its the same song as the requested one
        1. if yes, get different random song
        2. else, recommend song
3. if song not in playlist, submit song to spotify for audio features

In [96]:
# libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass

# LOGIN
client_id = str(getpass.getpass('client_id?'))
client_secret = str(getpass.getpass('client_secret?'))

sp=spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id = client_id, client_secret= client_secret))


# formats the dataframe - lowercases and removes special characters and spaces from df
def format_df(df):
    df_lower = df.applymap(lambda s: s.lower() if type(s) == str else s)
    formatter = lambda s: ''.join(e for e in s if e.isalnum())
    df_clean = df_lower.applymap(formatter)
    return df



# function which handles more than 100 results
def get_playlist_tracks(user_id, playlist_id):
    
    # LOGIN
    #client_id = str(getpass.getpass('client_id?'))
    #client_secret = str(getpass.getpass('client_secret?'))
    client_id = '656c9257e8044fe98858ea5f578096bd'
    client_secret = '9b459f6afdf244168d86bac6cce0a90e'
    sp=spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id = client_id, client_secret= client_secret))
    
    # Query Spotify
    print('Fetching songs from Spotify...')
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks



# gets songs from chosen spotify playlist
def fetch_hot_songs_spotify():
        
    # specify playlists
    #user_id = 'maka_97'
    #playlist_id = '6mtYuOxzl58vSGnEDtZ9uB'
    user_id = '12174441449'
    playlist_id = '1T2VEWeXUzUzZ6piswWBjZ'
    
    # get songs from playlist
    playlist = get_playlist_tracks(user_id, playlist_id)
     
    # append songs and artists to lists
    song_ids = [r["track"]["id"] for r in playlist]
    songs = [r["track"]["name"] for r in playlist]
    artists = [r["track"]["artists"][0]['name'] for r in playlist]
    # write to DataFrame
    songs_df = pd.DataFrame({'song_ids':song_ids, 'songs':songs, 'artists':artists})
    
    # break song_ids into chunks of 100
    blocks = [song_ids[x:x+100] for x in range(0, len(song_ids), 100)]
    
    # get audio features for fetched Spotify songs
    print('Fetching audio features...')
    # submit blocks to spotify and add results to list
    feats = []
    for x in tqdm(range(len(blocks))):
        feats.append(sp.audio_features(tracks= blocks[x])) # returns list with blocks of dicts
    
    # add each dict in each block to new list
    feat_dicts = []
    for block in feats:
        for feat_dict in block:
            feat_dicts.append(feat_dict)
     
    feats_df = pd.DataFrame(feat_dicts)
    
    # join dataframes 
    df = pd.concat([songs_df, feats_df], axis=1)
    
    # export to csv
    df.to_csv('TopSongs_Spotify.csv', index=False) 
    
    # write timestamp to txt file
    timestamp= datetime.today().strftime('%Y-%m-%d')
    with open('last_scrape_spotify.txt', 'w') as f:
        f.write(timestamp)
            
    print('Fetch completed!\n')
    return df



In [8]:
# import tools
from bs4 import BeautifulSoup
import pandas as pd
import requests
from tqdm.notebook import tqdm
from datetime import datetime
import random
from IPython.display import clear_output
from datetime import datetime

### Functions for search

In [2]:
   
# displays exit message
def say_goodbye():
    print('Thanks for using BEETz! Come back soon!')
    return



# checks for last scrape by reading txt file in which last date was stored
def check_last_scrape():
    with open('last_scrape.txt', 'r') as f:
        date = f.read()
    return date
        
    
    
# random excitement
def exclaim():
    exclamations=['SOO hot right now!', 'So hot its stolen!', 'Sizzzzzling!']
    exclamation = random.choice(exclamations)
    print('\n', exclamation)

    
    
# scrapes songs from online billboard, writes to csv, returns as df and records timestamp in txt file
def fetch_hot_songs():
    print('Fetching hot songs and artists from the web...')
    
    url = "https://www.billboard.com/charts/hot-100/"
    response = requests.get(url)
    website_data = response.text
    soup = BeautifulSoup(website_data, "html.parser")
    
    # append songs and artists to lists
    songs = []
    artists = []
    for i in tqdm(range(100)):
        songs.append(soup.select('h3.c-title.a-no-trucate')[i].get_text(strip=True))
        artists.append(soup.select('span.c-label.a-no-trucate')[i].get_text(strip=True))
    
    # write to DataFrame
    df_dirty = pd.DataFrame({'songs':songs, 'artists':artists})
    df.to_csv('TopSongs.csv', index=False) 
    
    # write timestamp to txt file
    timestamp= datetime.today().strftime('%Y-%m-%d')
    with open('last_scrape.txt', 'w') as f:
        f.write(timestamp)
            
    print('Fetch completed!\n')
    return df
    
    
    
# gets random song from scraped billboard and checks if identical with requested song
def get_random_song(x, df):
    # takes sample from unclean dataframe
    sample = df.sample()
    rec_song = str(sample.iloc[0, 0])
    rec_artist = str(sample.iloc[0, 1])
    # clean song and check to make sure its not the same song as requested
    lower_song = rec_song.lower()
    song_cleaned = ''.join(e for e in lower_song if e.isalnum())
    if x in song_cleaned:
        get_random_song(x, df_clean, df)
    else:
        print(f'BEETz recommends {rec_song} by {rec_artist}!')
        print('Go again?')
    
        

        
# Based on run_search, asks user to enter song, formats and checks to see if song is in 'hot' songs
def song_search(df_clean, df):
    song = input('Please enter the title of a song you enjoy: \n').lower()
    # remove spaces and special characters
    song_cleaned = ''.join(e for e in song if e.isalnum())
    
    # check if song title is in top hits
    if df_clean['songs'].str.contains(song_cleaned).any():
        exclaim()
        return song_cleaned
    # if not, offer another search
    else:
        print('Your song doesn\'t seem to be very popular...try again? ')
        #run_search(df_clean, df)
        
        '''ENTER CODE FOR SPOTIPY HERE''' 
        df = fetch_hot_songs_spotify()
    
# Based on run_search, asks user to enter artist, formats and checks if artist is in 'hot' artists
def artist_search(df_clean, df):
    artist = input('Please enter the name of an artist you enjoy: \n').lower()
    # remove spaces and special characters
    artist_cleaned = ''.join(e for e in artist if e.isalnum())
    
    # check if artist is in top artists
    if df_clean['artists'].str.contains(artist_cleaned).any():
        exclaim()
        return '-'
    # if not, offer another search
    else:
        print('Your artist doesn\'t seem to be very popular...try again? ')
        run_search(df_clean, df)
            

            
# gets user search preference and executes search accordingly
def run_search(df_clean, df):    
    response = input('''
    Enter 1 to search by song.
    Enter 2 to search by artist. 
    Enter 3 to exit.\n\n''')  
    if response == '1':
        x = song_search(df_clean, df)
    elif response == '2':
        x = artist_search(df_clean, df)
    elif response == '3':
        say_goodbye()
        return
    else:
        print('Hm...that didn\'t work. Please try again.')
        run_search(df_clean, df)
    
    # Recommends random song after checking that it doesn't match searched song
    get_random_song(x, df)
    
    run_search(df_clean, df)
        

## Primary function

In [3]:
# the MOTHER function

def beetz():
    
    print('''
     ____  ______ ______ _______  
 |  _ \|  ____|  ____|__   __| 
 | |_) | |__  | |__     | |____
 |  _ <|  __| |  __|    | |_  /
 | |_) | |____| |____   | |/ / 
 |____/|______|______|  |_/___|
 ''')
        
    print('HELLO! Welcome to the BEETz song recommender!')
    print('BEETz will recommend a song related to the song or artist you enter.')
    
    
    # checks for last scrape - if not from today, launches new scrape
    if check_last_scrape() == datetime.today().strftime('%Y-%m-%d'):
        print('We have an up-to-date list of HOT songs from today on file!')
        df = pd.read_csv('TopSongs.csv', index_col=False)
    else:
        print('We will fetch an up-to-date list of HOT songs from today!')
        df = fetch_hot_songs()
    
    # lowercase and remove special characters and spaces from df
    df_lower = df.applymap(lambda s: s.lower() if type(s) == str else s)
    formatter = lambda s: ''.join(e for e in s if e.isalnum())
    df_clean = df_lower.applymap(formatter)
    
    # gets user search preference and executes search accordingly
    run_search(df_clean, df)
    

In [4]:
beetz()


     ____  ______ ______ _______  
 |  _ \|  ____|  ____|__   __| 
 | |_) | |__  | |__     | |____
 |  _ <|  __| |  __|    | |_  /
 | |_) | |____| |____   | |/ / 
 |____/|______|______|  |_/___|
 
HELLO! Welcome to the BEETz song recommender!
BEETz will recommend a song related to the song or artist you enter.
We have an up-to-date list of HOT songs from today on file!

    Enter 1 to search by song.
    Enter 2 to search by artist. 
    Enter 3 to exit.

3
Thanks for using BEETz! Come back soon!


In [97]:
# fetch hot songs from spotify
fetch_hot_songs_spotify()


Fetching songs from Spotify...
Fetching audio features...


  0%|          | 0/19 [00:00<?, ?it/s]

Fetch completed!



,song_ids,songs,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,2I6mf3RFLj1c0k5OpX5sKg,Guns and Dogs,Portugal. The Man,0.614,0.670,10,-6.616,1,0.0329,0.360000,...,0.1080,0.578,80.022,audio_features,2I6mf3RFLj1c0k5OpX5sKg,spotify:track:2I6mf3RFLj1c0k5OpX5sKg,https://api.spotify.com/v1/tracks/2I6mf3RFLj1c...,https://api.spotify.com/v1/audio-analysis/2I6m...,164520,4
1,6QgjcU0zLnzq5OrUoSZ3OK,Feel It Still,Portugal. The Man,0.801,0.795,1,-5.115,0,0.0504,0.041700,...,0.0717,0.754,79.028,audio_features,6QgjcU0zLnzq5OrUoSZ3OK,spotify:track:6QgjcU0zLnzq5OrUoSZ3OK,https://api.spotify.com/v1/tracks/6QgjcU0zLnzq...,https://api.spotify.com/v1/audio-analysis/6Qgj...,163253,4
2,4SNtyKNyYJ8ERT7YLlnFBr,So Young,Portugal. The Man,0.671,0.445,4,-7.842,0,0.0291,0.348000,...,0.1550,0.811,75.996,audio_features,4SNtyKNyYJ8ERT7YLlnFBr,spotify:track:4SNtyKNyYJ8ERT7YLlnFBr,https://api.spotify.com/v1/tracks/4SNtyKNyYJ8E...,https://api.spotify.com/v1/audio-analysis/4SNt...,246480,4
3,0NBAXi7pWtoiLo7oomWBKw,Forever & Ever More,Nothing But Thieves,0.289,0.841,8,-4.330,1,0.1460,0.005820,...,0.2670,0.399,183.170,audio_features,0NBAXi7pWtoiLo7oomWBKw,spotify:track:0NBAXi7pWtoiLo7oomWBKw,https://api.spotify.com/v1/tracks/0NBAXi7pWtoi...,https://api.spotify.com/v1/audio-analysis/0NBA...,208134,3
4,3meXdlcyYTgukuOfX6VyhJ,Amsterdam,Nothing But Thieves,0.513,0.932,10,-5.016,1,0.1720,0.004560,...,0.2450,0.665,168.062,audio_features,3meXdlcyYTgukuOfX6VyhJ,spotify:track:3meXdlcyYTgukuOfX6VyhJ,https://api.spotify.com/v1/tracks/3meXdlcyYTgu...,https://api.spotify.com/v1/audio-analysis/3meX...,272240,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,27rgTetikreqkvedaxrF5N,Grounds for Divorce,Elbow,0.526,0.756,2,-5.083,0,0.0266,0.400000,...,0.4260,0.668,91.985,audio_features,27rgTetikreqkvedaxrF5N,spotify:track:27rgTetikreqkvedaxrF5N,https://api.spotify.com/v1/tracks/27rgTetikreq...,https://api.spotify.com/v1/audio-analysis/27rg...,219453,4
1857,1yaW0E2xUegutCOM316pup,Woke Up This Morning,Alabama 3,0.609,0.763,0,-9.955,0,0.1450,0.147000,...,0.0840,0.768,176.981,audio_features,1yaW0E2xUegutCOM316pup,spotify:track:1yaW0E2xUegutCOM316pup,https://api.spotify.com/v1/tracks/1yaW0E2xUegu...,https://api.spotify.com/v1/audio-analysis/1yaW...,316600,4
1858,4OSWIup9gKCkIdVvdkLEOE,Mercy,Ayron Jones,0.237,0.946,10,-4.056,0,0.1280,0.000149,...,0.1670,0.284,159.262,audio_features,4OSWIup9gKCkIdVvdkLEOE,spotify:track:4OSWIup9gKCkIdVvdkLEOE,https://api.spotify.com/v1/tracks/4OSWIup9gKCk...,https://api.spotify.com/v1/audio-analysis/4OSW...,205831,4
1859,2TmVlVk2t6d9MoqLNasfgS,Take Me Away,Ayron Jones,0.474,0.850,3,-4.439,0,0.0607,0.000091,...,0.2250,0.572,156.267,audio_features,2TmVlVk2t6d9MoqLNasfgS,spotify:track:2TmVlVk2t6d9MoqLNasfgS,https://api.spotify.com/v1/tracks/2TmVlVk2t6d9...,https://api.spotify.com/v1/audio-analysis/2TmV...,246933,4
